In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import splitfolders
import sys
import pathlib
import pandas as pd
import pickle

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
import tensorflow_datasets as tfds

from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import KFold, StratifiedKFold

from helper_functions import print_sens_spec_3class, print_sens_spec_2class

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
print(sys.version)
print('tensorflow: %s' % tf.__version__)
print('keras: %s' % keras.__version__)

In [ ]:
# Define parameters
batch_size = 32
img_height = 256
img_width = 256
num_splits = 5
binary_threshold = 0.5
epochs = 35
learning_rate=0.001

class_names = ['Normal','DME']
num_classes = len(class_names)

## Read Data and Train Model

In [ ]:
trainval_dir = "Data/Primary-2-class-train-val"
trainval_dir = pathlib.Path(trainval_dir)

trainval_count = len(list(trainval_dir.glob('*/*.png')))
print("Training & Val Count: " + str(trainval_count))

In [ ]:
trainval_ds = tf.keras.utils.image_dataset_from_directory(
    trainval_dir,
    seed=124,
    color_mode='grayscale',
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='binary',
    class_names=class_names
)

In [ ]:
# Convert dataset to numpy for use with kfold
trainval_ds_ub = trainval_ds.unbatch()
trainval_ds_np = tfds.as_numpy(trainval_ds_ub)

trainval_data = []
trainval_labels = []

for entry in trainval_ds_np:
    trainval_data.append(entry[0])
    trainval_labels.append(entry[1])
    
trainval_data_np = np.array(trainval_data)
trainval_labels_np = np.array(trainval_labels)

print(trainval_data_np.shape)
print(trainval_labels_np.shape)

In [ ]:
# Define kfold split
kfold = StratifiedKFold(n_splits=num_splits, shuffle=True)

In [ ]:
# Data augmentation
with tf.device('/CPU:0'):

    data_augmentation = keras.Sequential(
      [
        layers.RandomFlip("horizontal",input_shape=(img_height,img_width,1),seed=136),
        layers.RandomRotation(0.1, seed=175),
        layers.RandomZoom(0.1, seed=181),
      ]
    )

In [ ]:
# Define Model
def getModel():
    
    model = Sequential([
        data_augmentation,
        layers.Rescaling(1./255, input_shape=(img_height, img_width, 1)),
        layers.Conv2D(16, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(256, 3, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.20),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    
    return model

In [ ]:
%%time
tf.keras.backend.clear_session()

loss_values = []
acc_scores = []
auc_scores = []
prec_scores = []
recall_scores = []

histories = []
models = []

val_predictions_values_cv = []
val_predictions_cv = []
val_labels_cv = []

i = 1

for train_index, val_index in kfold.split(trainval_data_np, trainval_labels_np):
    
    tf.keras.backend.clear_session()

    print("\n#### This is split: " + str(i) + " ####")
    
    train_data_fold, val_data_fold = trainval_data_np[train_index], trainval_data_np[val_index]
    train_labels_fold, val_labels_fold = trainval_labels_np[train_index], trainval_labels_np[val_index]
    
    print("Training Entries: " + str(train_data_fold.shape[0]))
    print("Validation Entries: " + str(val_data_fold.shape[0]))
    print("\n")
    
    model = getModel()
    
    opt = keras.optimizers.Adam(learning_rate=learning_rate)
    
    model.compile(optimizer=opt,
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(),
                       tf.keras.metrics.AUC(),
                       tf.keras.metrics.Precision(),
                       tf.keras.metrics.Recall()])

    history = model.fit(train_data_fold,train_labels_fold,
                        epochs=epochs,
                        batch_size = batch_size, 
                        validation_data = (val_data_fold, val_labels_fold))
    
    histories.append(history)

    scores = model.evaluate(val_data_fold, val_labels_fold)
    
    v_predictions_values = model.predict(val_data_fold)
    v_predictions = np.where(v_predictions_values > binary_threshold, 1, 0)
    v_labels = val_labels_fold
    
    val_predictions_values_cv.append(v_predictions_values)
    val_predictions_cv.append(v_predictions)
    val_labels_cv.append(v_labels)

    print("%s: %.2f%% \n" % (model.metrics_names[1], scores[1]*100))

    loss_values.append(scores[0])
    acc_scores.append(scores[1])
    auc_scores.append(scores[2])
    prec_scores.append(scores[3])
    recall_scores.append(scores[4])
    i += 1

print("\n#### OVERALL SUMMARY - FROM EVALUATION SETS ####")
print("Average final loss: %.2f (+/- %.2f)" % (np.mean(loss_values), np.std(loss_values)))
print("Average accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(acc_scores)*100, np.std(acc_scores)*100))
print("Average final AUC: %.2f (+/- %.2f)" % (np.mean(auc_scores), np.std(auc_scores)))
print("Average precision: %.2f%% (+/- %.2f%%)" % (np.mean(prec_scores)*100, np.std(prec_scores)*100))
print("Average recall: %.2f%% (+/- %.2f%%)" % (np.mean(recall_scores)*100, np.std(recall_scores)*100))

In [ ]:
# Record to check epoch for minimum loss and max accuracy
loss_val_values = np.zeros(epochs)
acc_val_values = np.zeros(epochs)

for history in histories:
    
    epochs_plot = range(1,epochs+1)

    # Training and Validation Accuracy and Loss
    train_acc = history.history['binary_accuracy']
    val_acc = history.history['val_binary_accuracy']
    
    acc_val_values = np.add(acc_val_values, val_acc)

    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    loss_val_values = np.add(loss_val_values, val_loss)

    train_prec = history.history['precision']
    val_prec = history.history['val_precision']

    train_recall = history.history['recall']
    val_recall = history.history['val_recall']

    train_auc = history.history['auc']
    val_auc = history.history['val_auc']

    epochs_range = range(epochs)

    plt.figure(figsize=(20, 8))
    plt.subplot(1, 5, 1)
    plt.plot(epochs_plot, train_acc, label='Training Accuracy')
    plt.plot(epochs_plot, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 5, 2)
    plt.plot(epochs_plot, train_loss, label='Training Loss')
    plt.plot(epochs_plot, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')

    plt.subplot(1, 5, 3)
    plt.plot(epochs_plot, train_prec, label='Precision')
    plt.plot(epochs_plot, val_prec, label='Validation Precision')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Precision')

    plt.subplot(1, 5, 4)
    plt.plot(epochs_plot, train_recall, label='Recall')
    plt.plot(epochs_plot, val_recall, label='Validation Recall')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Recall')

    plt.subplot(1, 5, 5)
    plt.plot(epochs_plot, train_auc, label='AUC')
    plt.plot(epochs_plot, val_auc, label='Validation AUC')
    plt.legend(loc='lower right')
    plt.title('Training and Validation AUC')

    plt.show()

In [ ]:
avg_loss_val_values = loss_val_values/num_splits
min_avg_loss = min(avg_loss_val_values)
min_loss_epoch = (np.argmin(avg_loss_val_values) + 1)

print("Average minimum loss: " + str(min_avg_loss))
print("Epoch number for min loss: " + str(min_loss_epoch))

epochs_plot = range(1,epochs+1)

plt.figure()
plt.plot(epochs_plot,avg_loss_val_values,)
plt.xlabel('Epoch')
plt.ylabel('Avg Loss')
plt.title('Loss over epochs')

In [ ]:
avg_acc_val_values = acc_val_values/num_splits
max_avg_acc = max(avg_acc_val_values)
max_acc_epoch = (np.argmax(avg_acc_val_values) + 1)

print("Average maximum accuracy: " + str(max_avg_acc))
print("Epoch number for max acc: " + str(max_acc_epoch))

epochs_plot = range(1,epochs+1)

plt.figure()
plt.plot(epochs_plot,avg_acc_val_values,)
plt.xlabel('Epoch')
plt.ylabel('Avg Accuracy')
plt.title('Accuracy over epochs')

## Metrics on the Cross Validation Sets

In [ ]:
for i in range(num_splits):

    val_labels = val_labels_cv[i]
    val_predictions = val_predictions_cv[i]
    val_predictions_values = val_predictions_values_cv[i]
    
    print("\n### Results for split number: " + str(i+1) + "\n")

    print(classification_report(val_labels, val_predictions, target_names=['Normal','DME']))

    cm = confusion_matrix(val_labels, val_predictions, labels=[0,1])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=['Normal','DME'])
    disp.plot()
    
    print_sens_spec_2class(cm)
    
    fpr_val = dict()
    tpr_val = dict()
    roc_auc_val = dict()
    
    plt.figure()

    fpr_val, tpr_val, thresholds = roc_curve(val_labels, val_predictions_values)
    roc_auc_val = auc(fpr_val, tpr_val)
    plt.plot(fpr_val, tpr_val, label='ROC curve (area = {0:0.2f})'.format(roc_auc_val))

    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic - Validation Data')
    plt.legend(loc="lower right")
    plt.show()    

## Retrain the dataset on all of the data

In [ ]:
trainval_ds = tf.keras.utils.image_dataset_from_directory(
    trainval_dir,
    seed=124,
    color_mode='grayscale',
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='binary',
    class_names=class_names
)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
trainval_ds_p = trainval_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

In [ ]:
full_model = getModel()
    
opt = keras.optimizers.Adam(learning_rate=learning_rate)

full_model.compile(optimizer=opt,
          loss=tf.keras.losses.BinaryCrossentropy(),
          metrics=[tf.keras.metrics.BinaryAccuracy(),
                   tf.keras.metrics.AUC(),
                   tf.keras.metrics.Precision(),
                   tf.keras.metrics.Recall()])

history = full_model.fit(trainval_ds_p,
                         epochs=epochs,
                         batch_size = batch_size)

In [ ]:
# Save the model
# full_model.save("Models/model_name.h5")

In [ ]:
# Load the model 
# full_model = keras.models.load_model("Models/model_name.h5")

In [ ]:
full_model.summary()

## Check Test Dataset - First Ophthalmologist

In [ ]:
test_dir_first = "Data/Test-2-class-1st-ophth"

test_ds_first = tf.keras.utils.image_dataset_from_directory(
    test_dir_first,
    shuffle=False,
    color_mode='grayscale',
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='binary',
    class_names=class_names
)

In [ ]:
full_model.evaluate(test_ds_first, batch_size=batch_size)

In [ ]:
# Get predictions and get evaluation
test_predictions_values_first = full_model.predict(test_ds_first)
test_predictions_first = np.where(test_predictions_values_first > binary_threshold, 1,0)
test_labels_first = np.concatenate(list(test_ds_first.map(lambda x, y:y)))

print(classification_report(test_labels_first, test_predictions_first, target_names=class_names))

In [ ]:
cm_first = confusion_matrix(test_labels_first, test_predictions_first, labels=[0,1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm_first,display_labels=class_names)

disp.plot(cmap=plt.cm.Blues)
disp.ax_.set_title('1st Ophthalmologist')

In [ ]:
# disp.figure_.savefig('Figures/cm-2class-test-first-ophth.pdf', bbox_inches = 'tight')

In [ ]:
print_sens_spec_2class(cm_first)

In [ ]:
# Examine predictions for the validation data ~ print percentages for each entry and consider effect of model
test_filenames_first = [l[-10:-4] for l in test_ds_first.file_paths]

In [ ]:
df_test_first = pd.DataFrame(test_filenames_first, columns=['Filename'])
df_test_first['DME-Prob'] = test_predictions_values_first[:,0]
df_test_first['Predicted'] = test_predictions_first
df_test_first['Actual'] = test_labels_first
class_codes = {0:'Normal', 1:'DME'}
df_test_first['Predicted'] = df_test_first['Predicted'].map(class_codes)
df_test_first['Actual'] = df_test_first['Actual'].map(class_codes)
df_test_first['Correct'] = df_test_first['Predicted'] == df_test_first['Actual']

In [ ]:
pd.set_option("display.max_rows", None)
pd.options.display.float_format = '{:,.4f}'.format
display(df_test_first)

In [ ]:
# Compute ROC curve
plt.figure()

fpr_test_first, tpr_test_first, thresholds = roc_curve(test_labels_first, test_predictions_values_first)
roc_auc_test_first = auc(fpr_test_first, tpr_test_first)
plt.plot(fpr_test_first, tpr_test_first, label='ROC curve (area = {0:0.2f})'.format(roc_auc_test_first))

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('1st Ophthalmologist')
plt.legend(loc="lower right")

# plt.savefig('figures/ROC-2class-test-first-ophth.pdf', bbox_inches = 'tight')

plt.show()

In [ ]:
roc_auc_test_first

## Check Test Dataset - Second Ophthalmologist

In [ ]:
test_dir_second = "Data/Test-2-class-2nd-ophth"

test_ds_second = tf.keras.utils.image_dataset_from_directory(
    test_dir_second,
    shuffle=False,
    color_mode='grayscale',
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='binary',
    class_names=class_names
)

In [ ]:
full_model.evaluate(test_ds_second, batch_size=batch_size)

In [ ]:
# Get predictions and get evaluation
test_predictions_values_second = full_model.predict(test_ds_second)
test_predictions_second = np.where(test_predictions_values_second > binary_threshold, 1,0)
test_labels_second = np.concatenate(list(test_ds_second.map(lambda x, y:y)))

print(classification_report(test_labels_second, test_predictions_second, target_names=class_names))

In [ ]:
cm_second = confusion_matrix(test_labels_second, test_predictions_second, labels=[0,1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm_second,display_labels=class_names)

disp.plot(cmap=plt.cm.Blues)
disp.ax_.set_title('2nd Ophthalmologist')

In [ ]:
# disp.figure_.savefig('figures/cm-2class-test-second-ophth.pdf', bbox_inches = 'tight')

In [ ]:
print_sens_spec_2class(cm_second)

In [ ]:
# Examine predictions for the validation data ~ print percentages for each entry and consider effect of model
test_filenames_second = [l[-10:-4] for l in test_ds_second.file_paths]

In [ ]:
df_test_second = pd.DataFrame(test_filenames_second, columns=['Filename'])
df_test_second['DME-Prob'] = test_predictions_values_second[:,0]
df_test_second['Predicted'] = test_predictions_second
df_test_second['Actual'] = test_labels_second
class_codes = {0:'Normal', 1:'DME'}
df_test_second['Predicted'] = df_test_second['Predicted'].map(class_codes)
df_test_second['Actual'] = df_test_second['Actual'].map(class_codes)
df_test_second['Correct'] = df_test_second['Predicted'] == df_test_second['Actual']

In [ ]:
pd.set_option("display.max_rows", None)
pd.options.display.float_format = '{:,.4f}'.format
display(df_test_second)

In [ ]:
# Compute ROC curve
plt.figure()

fpr_test_second, tpr_test_second, thresholds = roc_curve(test_labels_second, test_predictions_values_second)
roc_auc_test_second = auc(fpr_test_second, tpr_test_second)
plt.plot(fpr_test_second, tpr_test_second, label='ROC curve (area = {0:0.2f})'.format(roc_auc_test_second))

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('2nd Ophthalmologist')
plt.legend(loc="lower right")

# plt.savefig('figures/ROC-2class-test-second-ophth.pdf', bbox_inches = 'tight')

plt.show()

In [ ]:
roc_auc_test_second

## Run for External Dataset - Kermany 2018 DME and NORMAL Data only

In [ ]:
ext_dir = "Data/Extern-Kermany-2018"

ext_ds = tf.keras.utils.image_dataset_from_directory(
    ext_dir,
    shuffle=False,
    color_mode='grayscale',
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='binary',
    class_names=class_names
)

In [ ]:
full_model.evaluate(ext_ds, batch_size=batch_size)

In [ ]:
# Get predictions and get evaluation
ext_predictions_values = full_model.predict(ext_ds)
ext_predictions = np.where(ext_predictions_values > binary_threshold, 1,0)
ext_labels = np.concatenate(list(ext_ds.map(lambda x, y:y)))

print(classification_report(ext_labels, ext_predictions, target_names=class_names))

In [ ]:
ext_cm = confusion_matrix(ext_labels, ext_predictions, labels=[0,1])
disp = ConfusionMatrixDisplay(confusion_matrix=ext_cm,display_labels=class_names)

disp.plot(cmap=plt.cm.Blues)
disp.ax_.set_title('External Dataset')

In [ ]:
# disp.figure_.savefig('figures/cm-2class-kermany-ext-test.pdf', bbox_inches = 'tight')

In [ ]:
print_sens_spec_2class(ext_cm)

In [ ]:
# Examine predictions for the validation data ~ print percentages for each entry and consider effect of model
ext_filenames = [l[-19:-5] for l in ext_ds.file_paths]

In [ ]:
df_ext = pd.DataFrame(ext_filenames, columns=['Filename'])
df_ext['DME-Prob'] = ext_predictions_values[:,0]
df_ext['Predicted'] = ext_predictions
df_ext['Actual'] = ext_labels
class_codes = {0:'Normal', 1:'DME'}
df_ext['Predicted'] = df_ext['Predicted'].map(class_codes)
df_ext['Actual'] = df_ext['Actual'].map(class_codes)
df_ext['Correct'] = df_ext['Predicted'] == df_ext['Actual']

In [ ]:
pd.set_option("display.max_rows", None)
pd.options.display.float_format = '{:,.4f}'.format
display(df_ext)

In [ ]:
# Compute ROC curve
plt.figure()

fpr_ext, tpr_ext, thresholds = roc_curve(ext_labels, ext_predictions_values)
roc_auc_ext = auc(fpr_ext, tpr_ext)
plt.plot(fpr_ext, tpr_ext, label='ROC curve (area = {0:0.2f})'.format(roc_auc_ext))

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('External Dataset')
plt.legend(loc="lower right")

# plt.savefig('figures/ROC-2class-kermany-ext-test.pdf', bbox_inches = 'tight')

plt.show()

In [ ]:
roc_auc_ext